In [7]:
pip install pygame


   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   -----------------


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import pygame
import heapq
import math
import random

# Initialize pygame, which handles the graphical interface
pygame.init()

# Define screen dimensions and grid properties
WIDTH = 800       # Width of the window
HEIGHT = 800      # Height of the window
ROWS = 50         # Number of rows in the grid
COLS = 50         # Number of columns in the grid
CELL_WIDTH = WIDTH // COLS     # Width of each cell in the grid
CELL_HEIGHT = HEIGHT // ROWS   # Height of each cell in the grid

# Define colors (RGB format) to use for different parts of the grid
WHITE = (255, 255, 255)     # Empty cells
BLACK = (0, 0, 0)          # Barriers
GREEN = (0, 255, 0)        # Start point
RED = (255, 0, 0)          # End point
BLUE = (0, 0, 255)         # Path found
GREY = (220, 220, 220)     # Grid lines

# Initialize the pygame window (screen) where the grid will be displayed
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Dijkstra's Algorithm - Pathfinding")   # Window title

# Class to represent each node (or cell) in the grid
class Node:
    def __init__(self, row, col):
        # Initialize node's position, color, neighbors, cost, and previous node
        self.row = row
        self.col = col
        self.color = WHITE     # Default color is white (empty)
        self.neighbors = []    # List to store adjacent nodes
        self.g_cost = float('inf')  # Distance cost from the start node (initially infinity)
        self.prev = None       # Previous node in the path (used to trace back the path)

    # Draws the node (cell) on the screen as a rectangle with its respective color
    def draw(self, screen):
        pygame.draw.rect(screen, self.color, (self.col * CELL_WIDTH, self.row * CELL_HEIGHT, CELL_WIDTH, CELL_HEIGHT))

    # Change the node color to represent the start point
    def set_start(self):
        self.color = GREEN

    # Change the node color to represent the end point
    def set_end(self):
        self.color = RED

    # Change the node color to represent a barrier
    def set_barrier(self):
        self.color = BLACK

    # Change the node color to represent a part of the found path
    def set_path(self):
        self.color = BLUE

    # Checks if the node is a barrier
    def is_barrier(self):
        return self.color == BLACK

    # Reset the node back to its default white color
    def reset(self):
        self.color = WHITE

# Initialize the grid as a 2D list of Node objects (50x50 grid)
grid = [[Node(i, j) for j in range(COLS)] for i in range(ROWS)]

# Dijkstra's Algorithm implementation
def dijkstra(start, end):
    count = 0
    # Priority queue to store nodes to explore (using heapq)
    pq = [(0, count, start)]
    start.g_cost = 0  # Set the start node's cost to 0 (since it's the starting point)
    visited = set()   # Set to keep track of visited nodes

    # Continue until the priority queue is empty
    while pq:
        current_node = heapq.heappop(pq)[2]  # Get the node with the smallest cost (from priority queue)

        if current_node == end:  # If we reached the end node, retrace the path
            return retrace_path(start, end)

        visited.add(current_node)  # Mark current node as visited

        # Loop through each neighbor of the current node
        for neighbor in current_node.neighbors:
            # Skip the neighbor if it's already visited or a barrier
            if neighbor in visited or neighbor.is_barrier():
                continue

            # Update the cost to reach this neighbor (since grid edges are uniform, cost is +1)
            new_g_cost = current_node.g_cost + 1
            if new_g_cost < neighbor.g_cost:  # If the new cost is smaller, update the neighbor's cost
                neighbor.g_cost = new_g_cost
                neighbor.prev = current_node  # Set the current node as the neighbor's previous node
                count += 1
                heapq.heappush(pq, (neighbor.g_cost, count, neighbor))  # Add the neighbor to the priority queue

    return False  # If no path is found, return False

# Function to retrace the path from the end node to the start node
def retrace_path(start, end):
    current = end
    while current.prev:  # Follow the 'prev' pointers to go backward through the path
        current = current.prev
        if current != start:  # Set the path color only if the node isn't the start node
            current.set_path()
    return True

# Function to get the neighbors (adjacent nodes) of a node
def get_neighbors(node):
    neighbors = []
    if node.row > 0:  # Up neighbor
        neighbors.append(grid[node.row - 1][node.col])
    if node.row < ROWS - 1:  # Down neighbor
        neighbors.append(grid[node.row + 1][node.col])
    if node.col > 0:  # Left neighbor
        neighbors.append(grid[node.row][node.col - 1])
    if node.col < COLS - 1:  # Right neighbor
        neighbors.append(grid[node.row][node.col + 1])
    return neighbors

# Add neighbors to each node in the grid (using the get_neighbors function)
for row in grid:
    for node in row:
        node.neighbors = get_neighbors(node)

# Function to draw the entire grid (loop through every node and call its draw() method)
def draw_grid(screen):
    for row in grid:
        for node in row:
            node.draw(screen)

# Function to randomly generate barriers (20% chance each node becomes a barrier)
def generate_random_barriers():
    for row in grid:
        for node in row:
            if random.random() < 0.2:  # 20% chance to be a barrier
                node.set_barrier()

# Main game loop
def main():
    start_node = None  # Start node initialization (None means not selected yet)
    end_node = None    # End node initialization
    running = True     # Game loop condition
    started = False    # Pathfinding started condition

    generate_random_barriers()  # Generate random barriers in the grid at the beginning

    while running:  # Game loop
        screen.fill(WHITE)  # Fill the screen with white color (background)
        draw_grid(screen)   # Draw the grid

        for event in pygame.event.get():
            if event.type == pygame.QUIT:  # Quit the game if the close button is pressed
                running = False

            if pygame.mouse.get_pressed()[0]:  # Left mouse button click
                pos = pygame.mouse.get_pos()   # Get the mouse position
                row = pos[1] // CELL_HEIGHT    # Calculate the row based on the position
                col = pos[0] // CELL_WIDTH     # Calculate the column
                node = grid[row][col]          # Get the corresponding node in the grid

                if not start_node:  # If start node hasn't been selected yet
                    start_node = node
                    start_node.set_start()  # Set the selected node as the start node

                elif not end_node and node != start_node:  # If end node hasn't been selected yet
                    end_node = node
                    end_node.set_end()  # Set the selected node as the end node

            elif pygame.mouse.get_pressed()[2]:  # Right mouse button click to reset a node
                pos = pygame.mouse.get_pos()     # Get mouse position
                row = pos[1] // CELL_HEIGHT      # Calculate the row based on position
                col = pos[0] // CELL_WIDTH       # Calculate the column
                node = grid[row][col]            # Get the corresponding node
                node.reset()                     # Reset the node to white (default)

                if node == start_node:  # If the reset node is the start node, reset the start
                    start_node = None
                elif node == end_node:  # If the reset node is the end node, reset the end
                    end_node = None

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE and start_node and end_node:
                    # Start the Dijkstra's algorithm if space is pressed and start/end are selected
                    for row in grid:
                        for node in row:
                            node.neighbors = get_neighbors(node)  # Update the neighbors
                    dijkstra(start_node, end_node)  # Run Dijkstra's algorithm

                if event.key == pygame.K_c:  # Reset the grid if 'C' is pressed
                    start_node = None
                    end_node = None
                    grid[:] = [[Node(i, j) for j in range(COLS)] for i in range(ROWS)]  # Recreate grid
                    generate_random_barriers()  # Generate new random barriers

        pygame.display.update()  # Update the screen with the new drawing

    pygame.quit()  # Quit the pygame module

# Call the main function to start the program
main()
